
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# LAB - Assembling a RAG Application

In this lab, we will assemble a Retrieval-augmented Generation (RAG) application using the components we previously created. The primary goal is to create a seamless pipeline where users can ask questions, and our system retrieves relevant documents from a Vector Search index to generate informative responses.


**Lab Outline:**

In this lab, you will need to complete the following tasks;

* **Task 1 :** Setup the Retriever Component

* **Task 2 :** Setup the Foundation Model

* **Task 3 :** Assemble the Complete RAG Solution

* **Task 4 :** Save the Model to Model Registry in Unity Catalog

**📝 Your task:** Complete the **`<FILL_IN>`** sections in the code blocks and follow the other steps as instructed.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **15.4.x-cpu-ml-scala2.12**

**🚨 Important:** This lab relies on the resources established in the previous one. Please ensure you have completed the prior lab before starting this one.


## Classroom Setup

Before starting the lab, run the provided classroom setup script. This script will define configuration variables necessary for the lab. Execute the following cell:

In [0]:
%pip install -U -qq databricks-vectorsearch langchain==0.3.7 flashrank langchain-databricks PyPDF2
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.63 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.11.7 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-04

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser11131632_1754970105@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser11131632_1754970105
Working Directory: /Volumes/dbacademy/ops/labuser11131632_1754970105@vocareum_com
Dataset Location:  NestedNamespace (arxiv='/Volumes/dbacademy_arxiv/v01', dais='/Volumes/dbacademy_dais/v01', news='/Volumes/dbacademy_news/v01', docs='/Volumes/dbacademy_docs/v01')


## Task 1: Setup the Retriever Component
**Steps:**
1. Define the embedding model.
1. Get the vector search index that was created in the previous lab.
1. Generate a **retriever** from the vector store. The retriever should return **three results.**
1. Write a test prompt and show the returned search results.


In [0]:
## Components we created before
vs_endpoint_prefix = "vs_endpoint_"
vs_endpoint_name = vs_endpoint_prefix+str(get_fixed_integer(DA.unique_name("_")))
print(f"Assigned Vector Search endpoint name: {vs_endpoint_name}.")

vs_index_fullname = f"{DA.catalog_name}.{DA.schema_name}.lab_pdf_text_managed_vs_index"

Assigned Vector Search endpoint name: vs_endpoint_2.


In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_databricks import DatabricksEmbeddings
from langchain_core.runnables import RunnableLambda
from langchain.docstore.document import Document
from flashrank import Ranker, RerankRequest

def get_retriever(cache_dir=f"{DA.paths.working_dir}/opt"):

    def retrieve(query, k: int=10):
        if isinstance(query, dict):
            query = next(iter(query.values()))

        ## get the vector search index
        vsc = VectorSearchClient(disable_notice=True)
        vs_index = vsc.get_index(endpoint_name=vs_endpoint_name, index_name=vs_index_fullname)
        
        ## get similar k documents
        return query, vs_index.similarity_search(
            query_text=query,
            columns=["pdf_name", "content"],
            num_results=k)

    def rerank(query, retrieved, cache_dir, k: int=2):
        ## format result to align with reranker lib format 
        passages = []
        for doc in retrieved.get("result", {}).get("data_array", []):
            new_doc = {"file": doc[0], "text": doc[1]}
            passages.append(new_doc)       
        ## Load the flashrank ranker
        ranker = Ranker(model_name="rank-T5-flan", cache_dir=cache_dir)

        ## rerank the retrieved documents
        rerankrequest = RerankRequest(query=query, passages=passages)
        results = ranker.rerank(rerankrequest)[:k]

        ## format the results of rerank to be ready for prompt
        return [Document(page_content=r.get("text"), metadata={"source": r.get("file")}) for r in results]

    ## the retriever is a runnable sequence of retrieving and reranking.
    return RunnableLambda(retrieve) | RunnableLambda(lambda x: rerank(x[0], x[1], cache_dir))

## test our retriever
question = {"input": "How does Generative AI impact humans?"}
vectorstore = get_retriever()
similar_documents = vectorstore.invoke(question)
print(f"Relevant documents: {similar_documents}")

Relevant documents: [Document(metadata={'source': 'dbfs:/Volumes/dbacademy_arxiv/v01/arxiv-articles/2303.10130.pdf'}, page_content='(Autoretal.,2022a)findsas\nwell that automation and augmentation exposures tend to be positively correlated. There is also a growing set\nof studies examining specific economic impacts and opportunities for LLMs (Bommasani et al., 2021; Felten\net al., 2023; Korinek, 2023; Mollick and Mollick, 2022; Noy and Zhang, 2023; Peng et al., 2023). Alongside\nthis work, our measurements help characterize the broader potential relevance of language models to the\nlabor market.\nGeneral-purpose technologies (e.g. printing, the steam engine) are characterized by widespread prolifera-\ntion, continuous improvement, and the generation of complementary innovations (Bresnahan and Trajtenberg,\n1995; Lipsey et al., 2005). Their far-reaching consequences, which unfold over decades, are difficult to\nanticipate,particularlyinrelationtolabordemand(Bessen,2018;KorinekandStigli

Trace(request_id=tr-6b8ca4c6646c442899d54b7d55a8b3f1)

## Task 2: Setup the Foundation Model
**Steps:**
1. Define the foundation model for generating responses. Use `llama-3.3` as foundation model. 
2. Test the foundation model to ensure it provides accurate responses.

In [0]:
## import necessary libraries
from langchain_databricks import ChatDatabricks

## define foundation model for generating responses
chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 300)

## test foundation model
print(f"Test chat model: {chat_model.invoke('What is Generative AI?')}")

/root/.ipykernel/8319/command-2179535239131770-3769911284:5: LangChainDeprecationWarning: Use databricks_langchain.ChatDatabricks
  chat_model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 300)


Test chat model: content='Generative AI refers to a type of artificial intelligence that is capable of generating new, original content, such as images, videos, music, text, or even entire datasets. This is in contrast to traditional AI, which is typically focused on analyzing and processing existing data.\n\nGenerative AI uses complex algorithms and machine learning techniques, such as deep learning and neural networks, to learn patterns and structures in data and then generate new data that is similar in style, structure, and content. The goal of generative AI is to create new, synthetic data that is indistinguishable from real data.\n\nThere are several types of generative AI, including:\n\n1. **Generative Adversarial Networks (GANs)**: GANs consist of two neural networks that work together to generate new data. One network generates new data, while the other network evaluates the generated data and tells the generator whether it is realistic or not.\n2. **Variational Autoencoders (

Trace(request_id=tr-f069a762817c490f9179ea45974e2763)

##Task 3: Assemble the Complete RAG Solution
**Steps:**
1. Merge the retriever and foundation model into a single Langchain chain.
2. Configure the Langchain chain with proper templates and context for generating responses.
3. Test the complete RAG solution with sample queries.

In [0]:
## import necessary libraries
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate

## define template for prompt
TEMPLATE = """You are an assistant for GENAI teaching class. You are answering questions related to Generative AI and how it impacts humans life. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
Use the following pieces of context to answer the question at the end:
{context}
Question: {input}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "input"])

## unwrap the longchain document from the context to be a dict so we can register the signature in mlflow
def unwrap_document(answer):
  return answer | {"context": [{"metadata": r.metadata, "page_content": r.page_content} for r in answer['context']]}

## merge retriever and foundation model into Langchain chain
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
chain = create_retrieval_chain(get_retriever(), question_answer_chain)|RunnableLambda(unwrap_document)


## test the complete RAG solution with sample query
question = {"input": "How Generative AI impacts humans?"}
answer = chain.invoke(question)
print(answer)

{'input': 'How Generative AI impacts humans?', 'context': [{'metadata': {'source': 'dbfs:/Volumes/dbacademy_arxiv/v01/arxiv-articles/2303.10130.pdf'}, 'page_content': 'arXivpreprint\narXiv:2302.07842.\nMoll,B.,Rachel,L.,andRestrepo,P.(2021). Unevengrowth: Automation’simpactonincomeandwealth\ninequality. SSRNElectronic Journal.\nMollick,E.R.andMollick,L.(2022). Newmodesoflearningenabledbyaichatbots: Threemethodsand\nassignments. Available atSSRN.\nNoy, S. and Zhang, W. (2023). Experimental evidence on the productivity effects of generative artificial\nintelligence. Available atSSRN4375283.\nO*NET (2023). O*net 27.2 database.\nOpenAI (2022). Introducing chatgpt.\nOpenAI (2023a). Gpt-4 system card. Technical report, OpenAI.\nOpenAI (2023b). Gpt-4 technical report. Technical report, OpenAI.\nOuyang,L.,Wu,J.,Jiang,X.,Almeida,D.,Wainwright,C.L.,Mishkin,P.,Zhang,C.,Agarwal,S.,Slama,\nK.,Ray,A.,etal.(2022). Traininglanguagemodelstofollowinstructionswithhumanfeedback. arXiv\npreprintarXiv:2203.

Trace(request_id=tr-403feadd0fbb466796e98edf95b4a2fe)

##Task 4: Save the Model to Model Registry in Unity Catalog
**Steps:**
1. Register the assembled RAG model in the Model Registry with Unity Catalog.
2. Ensure that all necessary dependencies and requirements are included.
3. Provide an input example and infer the signature for the model.

In [0]:
## import necessary libraries
from mlflow.models import infer_signature
import mlflow
import langchain

## set Model Registry URI to Unity Catalog
mlflow.set_registry_uri("databricks-uc")
model_name = f"{DA.catalog_name}.{DA.schema_name}.rag_app_lab_4"

## register the assembled RAG model in Model Registry with Unity Catalog
with mlflow.start_run(run_name="rag_app_lab_4") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
        ],
        input_example=question,
        signature=signature
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-385a80ad-7517-44df-9981-f963bf86b882/lib/python3.11/site-packages/langchain/chains/api/base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper
/local_disk0/.ephemeral_nfs/envs/pythonEnv-385a80ad-7517-44df-9981-f963bf86b882/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not 

Uploading artifacts:   0%|          | 0/36 [00:00<?, ?it/s]

Successfully registered model 'dbacademy.labuser11131632_1754970105.rag_app_lab_4'.


Uploading artifacts:   0%|          | 0/36 [00:00<?, ?it/s]

Created version '1' of model 'dbacademy.labuser11131632_1754970105.rag_app_lab_4'.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



## Clean up Resources

This was the final lab. You can delete all resources created in this course.


## Conclusion

In this lab, you learned how to assemble a Retrieval-augmented Generation (RAG) application using Databricks components. By integrating Vector Search for document retrieval and a foundational model for response generation, you created a powerful tool for answering user queries. This lab provided hands-on experience in building end-to-end AI applications and demonstrated the capabilities of Databricks for natural language processing tasks.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>